# Backpropagation by Hand
The API for this MLP network was inspired by sklearn.  
Derivations of update rules included but the network itself has a low accuracy for some reason, even with randomized weights, different activation functions etc.  
Only the Kullbach-Liebler divergence was attempted, and unfortunately, although the derivations seem accurate (matrix dimensions line up), the gradient is too small to move the weights. It is not known why.  

In [290]:
import numpy as np
import pickle
import random

In [308]:
""" Import training data and labels """
train_data = np.genfromtxt('./train_data.csv', delimiter=',')
train_labels = np.genfromtxt('./train_labels.csv', delimiter=',')

## Activation Functions
We include the following activation functions that can be used with the network.
- Sigmoid $S(x)=\frac{e^{x}}{1+e^{x}}$: a smooth and therefore differentiable alternative to the Signum function.

- Tanh $tanh(x)$: hyperbolic tan, similar to the sigmoid function but steeper around 0.

- ReLU $relu(x)=max\{0, x\}$: a non-linear ramp function

Also, the derivatives of these function are included.

In [309]:
""" Activation functions """

def sigmoid(x): # sigmoid activation
    """ sigmoid function """
    return 1/(np.exp(-x)+1)

def sigmoid_derivative(x):
    return sigmoid(x)*(1-sigmoid(x))

def tanh(x): # tanh activation
    """ tanh function"""
    return np.tanh(x)

def sec2h(x): # derivative of tanh
    cosh2x = np.power(np.cosh(x), 2)
    return 1/cosh2x

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return(np.heaviside(x, 0))


## Gradient Calculation Functions
Contains the evaluations of the derivatives necessary to compute the gradient of the weights.
Our network with activation function $\mathcal{F}$ will output the vector $$\vec{y}^{pred}=\mathcal{F}\bigg(\hat{W}^{O}\mathcal{F}\bigg(\hat{W}^{H}\vec{x}\bigg)\bigg)$$
Using a loss function $\mathcal{L}(\vec{y}^{pred}, \vec{y}_{target})$ we want to calculate the gradient with respect to the weights in the layer $i$ $\frac{\partial \mathcal{L}}{\partial \hat{W}^{i}}$ and then adjust the weights in this direction.  
Note: matrix calculus identities retrieved from here https://web.stanford.edu/class/cs224n/readings/gradient-notes.pdf

Since $$\vec{y}^{pred}=\mathcal{F}(\vec{u})=\mathcal{F}\bigg(\hat{W}^{O}\vec{O}^{H}\bigg)=\mathcal{F}\bigg(\hat{W}^{O}\mathcal{F}\big(\vec{v}\big)\bigg)=\mathcal{F}\bigg(\hat{W}^{O}\mathcal{F}\bigg(\hat{W}^{H}\vec{x}\bigg)\bigg)$$ we arrive at the following chain rule formulae for the gradients

$$\frac{\partial \mathcal{L}}{\partial \hat{W}^{H}}=\frac{\partial \mathcal{L}}{\partial \vec{O}^H}\frac{\partial \vec{O}^H}{\partial \hat{W}^H}=\bigg(\frac{\partial \mathcal{L}}{\partial \vec{O}^H}\bigg)^T\big(\vec{x}\big)^T=\big(\vec{\delta_1} \hat{\delta_2} \hat{\delta_3}\big)^T\big(\vec{x}\big)^T$$  
$$\frac{\partial \mathcal{L}}{\partial \hat{W}^{O}}=\frac{\partial \mathcal{L}}{\partial \vec{O}^O}\frac{\partial \vec{O}^O}{\partial \hat{W}^O}=\bigg(\frac{\partial \mathcal{L}}{\partial \vec{O}^O}\bigg)^T\big(\vec{O}^H\big)^T=\big(\vec{\delta_1} \hat{\delta_2}\big)^T\bigg(\vec{O}^H\bigg)^T$$
where we have introduced
$$\vec{\delta_1}=\frac{\partial \mathcal{L}}{\partial \vec{y}^{pred}}$$  
$$\hat{\delta_2}=\frac{\partial \vec{y}^{pred}}{\partial \vec{u}}=diag\bigg(\mathcal{F\prime}\bigg({\hat{W}^O\vec{O}^H}\bigg)\bigg)$$  
$$\hat{\delta_3}=\frac{\partial \vec{u}}{\partial \vec{O}^H}=\hat{W}^O$$

## Sigmoid Derivative
Therefore, for a sigmoid function:
$$\hat{\delta_2}=diag\bigg(S\bigg(\hat{W}^O\vec{O}^H\bigg)\big(1-S\bigg(\hat{W}^O\vec{O}^H\bigg)\big)\bigg)$$  

## Tanh Derivative
for the tanh function:
$$\hat{\delta_2}=diag\bigg(sech^2\big(\hat{W}^O\vec{O}^H\big)\bigg)$$

To calculate $\vec{\delta_1}$ we need to consider the loss function we are using:

### Kullbach-Liebler Divergence
The Kullbach-Liebler Divergence of two probability distributions $P$ and $Q$ over a sample space $\chi$ is given by $$\mathcal{L}_{KL}\Big( P|| Q \Big)=\sum_{x\in\chi}P(x)\log\bigg(\frac{P(x)}{Q(x)}\bigg)=\sum_{x\in\chi}P(x)\bigg(\log(P(x))-\log(Q(x))\bigg)$$ where $P$ is the reference distribution.  
In this case, we will take the one-hot encoded target values as the distribution $P$, and our softmaxed network output as the distribution $Q$.  
Using $\sigma$ for the softmax function on $\vec{y}^{pred}$, we have 
$$\mathcal{L}_{KL}=\sum_{i} y^{(targ)}_i\bigg(\log\big(y^{(targ)}_i\big)-\log(\vec{\sigma}_i)\bigg)$$  
$$\vec{\delta_1}=\frac{\partial \mathcal{L}_{KL}}{\partial \vec{y}^{pred}}=\frac{\partial \mathcal{L}_{KL}}{\partial \sigma}\frac{\partial \sigma}{\partial \vec{y}^{pred}}$$  
and the vector derivative of the softmax function is
$$\frac{\partial \vec{\sigma}}{\partial \vec{y}^{pred}}\bigg|_{i,j}=\begin{cases} \frac{\Bigg(\bigg(\sum_k \exp(y^{pred}_k)\bigg)-\exp(y^{pred}_{i})\Bigg)\exp(y^{pred}_{i})}{\bigg(\sum_k \exp(y^{pred}_k)\bigg)^2} &\mbox{if } i=j \\
\frac{-\exp(y^{pred}_{i})\exp(y^{pred}_{j})}{\bigg(\sum_k \exp(y^{pred}_k)\bigg)^2} & \mbox{if } i\neq j \end{cases}$$

## Utility Functions
Some useful functions for processing output.
- Softmax: converts neural network output into a probability distribution
- Softmax-to-one-hot: converts softmaxed data into one hot by setting the highest probability to 1 and the others to 0.

In [310]:
""" Utility functions """

def softmax(x_array): # softmax function
    """ Softmax function """
    return np.exp(x_array)/np.sum(np.exp(x_array))

def softmax_derivative(x_array):

    x_exp = np.exp(x_array)
    x_sum  = np.sum(np.exp(x_array))

    diag = lambda ex: (x_sum*ex-(ex**2))/(x_sum**2)
    offdiag = lambda ex1, ex2: -(ex1*ex2)/(x_sum**2)
    

    sigma_prime_matrix = np.array([[diag(x1) if i==j
                                    else offdiag(x1, x2)
                                    for j, x2 in enumerate(x_exp)]
                                   for i, x1 in enumerate(x_exp)])
    
    return sigma_prime_matrix
    

def softmax_to_one_hot(softmax_array): # convert softmax back to one hot
    """ Converts softmax vector to one hot encoded array """
    output = np.zeros(len(softmax_array), dtype='int')
    output[np.argmax(softmax_array)]=1
    return output


def train_test_split(inp_data, out_data, frac=0.7, random_seed=420):
    """ Shuffles a list of indices and returns split, randomized data """
    assert(len(inp_data)==len(out_data))
    
    idxs = np.arange(len(inp_data)) # generate list of indices
    random.Random(random_seed).shuffle(idxs) # randomly permute the list

    split_idx = round(len(inp_data)*frac) # get the first frac of the indices
    
    X_train, y_train = inp_data[:split_idx], out_data[:split_idx] # split the data
    X_test, y_test = inp_data[split_idx:], out_data[split_idx:]    # hold the test set

    return X_train, y_train, X_test, y_test


## Multilayer Perceptron Classes
We define a Node class to provide a neuron API for use by the network.  
The node is supplied with an activation function and input dimensions, and has a method for evaluating the function over the inputs, and updating the weights.

In [311]:
class Node:
    """ Generic Node class """
    input_weights = None    
    activation_function = None
    
    def __init__(self, input_dimension, activation_function):
        self.input_weights = np.random.rand(input_dimension)
        self.activation_function = activation_function

    def fire(self, input_vector):
        assert(len(input_vector) == len(self.input_weights))
        wTx = np.inner(self.input_weights, input_vector)
        return self.activation_function(wTx)

    def update_weights(self, new_weights):
        assert(new_weights.size == self.input_weights.size)
        self.input_weights = new_weights

The MLPNetwork class composes nodes into a network and provides an interface for interaction

In [321]:
class MLPNetwork:
    """ Class that composes Nodes into a NN and provides interface """
    num_hidden_nodes = None
    activation_function = None
    loss_function = None
    
    hidden_layer = None
    output_layer = None

    hidden_output = None
    network_output = None

    learning_rate = None
    
    network_created = False
    verbose = None
    
    def __init__(self, num_hidden_nodes, activation_function=sigmoid, loss_function='kldiv', verbose=True, learning_rate=0.1):
        assert(num_hidden_nodes>0)
        self.num_hidden_nodes = num_hidden_nodes
        self.activation_function = activation_function
        self.loss_function = loss_function
        self.learning_rate = learning_rate
        self.verbose = verbose

    def build_network(self, input_dimension, output_dimension):
        """ Builds the network from parameters """
        self.input_dimension = input_dimension
        self.output_dimension = output_dimension

        self.hidden_layer = [Node(self.input_dimension+1, self.activation_function)
                             for num in range(self.num_hidden_nodes)]

        """ Add bias neuron to hidden layer """
        self.hidden_layer.append(Node(self.input_dimension+1, self.activation_function))
        
        self.output_layer = [Node(self.num_hidden_nodes+1, self.activation_function)
                             for num in range(self.output_dimension)]


        self.network_created = True
        
    def delete_network(self):
        """ Deletes the network """
        self.hidden_layer = None
        self.output_layer = None
        self.network_created = False


    def training_epoch(self, X, y_target, point_idx=(None, None)):
        if (self.verbose and point_idx!=(None, None)):
            this_percentage = round(100*(point_idx[0]/point_idx[1]))
            print("[*] Training {0}%".format(this_percentage), end='\r')


        y_pred = self.feedforward(X)
        self.backpropagate(X, y_pred)

        return y_pred
        
    def train(self, X_train, y_train):
        if not self.network_created:
            self.build_network(X_train.shape[1], y_train.shape[1])

        training_set = zip(X_train, y_train)
        total = len(X_train)
            
        y_preds = np.array([self.training_epoch(X, y_target, (idx, total))
                            for idx, (X, y_target) in enumerate(training_set)])
        
    def read_weight_matrices(self):
        """ Read the weights from the layer nodes into matrices """

        self.hidden_weights = np.array([hidden_node.input_weights
                                        for hidden_node in self.hidden_layer])
    
    
        self.output_weights = np.array([output_node.input_weights
                                        for output_node in self.output_layer])
    
    def write_weight_matrices(self):
        """ Write a new weight matrix back to the nodes """
        [self.hidden_layer[idx].update_weights(weight_vector)
         for idx, weight_vector in enumerate(self.hidden_weights)]

        [self.output_layer[idx].update_weights(weight_vector)
         for idx, weight_vector in enumerate(self.output_weights)]

        
    def backpropagate(self, X, y):

        self.read_weight_matrices() # grab current weights

        if self.loss_function == 'test':

            #gradient_hidden =
            #gradient_output = 
            pass
        else:
            if self.loss_function == 'kldiv':
                """ row vector (row vector right multiplies matrix) """
                kl_deriv = y/softmax(self.network_output)
                sm_deriv = softmax_derivative(self.network_output)
                
                delta_1 = np.matmul(kl_deriv, sm_deriv)

            if self.activation_function == relu:
                """ matrix """
                delta_2 = np.diag(relu_derivative(self.output_weights @ self.hidden_output))

            elif self.activation_function == sigmoid:
                """ matrix """
                delta_2 = np.diag(self.network_output*(1-self.network_output))
            
            elif self.activation_function == tanh:
                """ matrix """
                delta_2 = np.diag(sec2h(self.output_weights @ self.hidden_output))
                
            """ matrix """
            delta_3 = self.output_weights
                
            """ matrix (row vector right multiplies matrix right multiplies matrix right multiplies matrix).T right multiplies (column vector).T """
            gradient_hidden = np.outer((delta_1 @ delta_2 @ delta_3).T, np.append(X, 1).T)

            """ matrix (row vector right multiplies matrix right multiplies matrix).T right multiplies (column vector).T """
            gradient_output = np.outer((delta_1 @ delta_2).T, (self.hidden_output).T)

        self.hidden_weights = self.hidden_weights-self.learning_rate*gradient_hidden
        self.output_weights = self.output_weights-self.learning_rate*gradient_output

        self.write_weight_matrices()
        
    
    def predict(self, X_test):
        
        output_vector = [self.feedforward(X_t) for X_t in X_test] 
        y_pred = np.array([softmax_to_one_hot(softmax(output)) for output in output_vector])
        return y_pred
    
    def feedforward(self, input_vector):
        """ Propagates a single datapoint with bias through the network """
        bias = 1
        self.hidden_output = np.array([hidden_node.fire(np.append(input_vector,bias)) for hidden_node in self.hidden_layer])
        self.network_output = np.array([output_node.fire(self.hidden_output) for output_node in self.output_layer])

        return self.network_output

    def import_config(self, filename='./mlpconfig'):
        config_file = pickle.load(open(filename, 'rb'))
        
        self.hidden_weights = config_file['h_weights']
        self.output_weights = config_file['o_weights']
        self.num_hidden_nodes = config_file['nhidden']
        self.loss_function = config_file['loss']
        
        self.build_network(config_file['idim'], config_file['odim'])
        self.write_weight_matrices()

    def export_config(self, filename='./mlpconfig'):
        cfg_dict = {'h_weights':self.hidden_weights,
                    'o_weights':self.output_weights,
                    'nhidden':self.num_hidden_nodes,
                    'loss':self.loss_function,
                    'idim':self.input_dimension,
                    'odim':self.output_dimension}
        pickle.dump(cfg_dict, open(filename, 'wb+'))
         

## Usage
Following the sklearn API, a network is initialized with various parameter values.  
The training set is loaded and split.

In [322]:
mlp_net = MLPNetwork(num_hidden_nodes=100,
                     activation_function=tanh,
                     loss_function='kldiv',
                     verbose=True,
                     learning_rate=0.1)


X_train, y_train, X_test, y_test = train_test_split(train_data, train_labels)

The network is trained via the train method, which takes a training and test set as arguments.  
Prediction is done via the predict method and takes X values as an argument.

In [324]:
mlp_net.train(X_train, y_train)
y_predictions = mlp_net.predict(X_test)

The configuration of the network can be exported with the method below, and imported by replacing export with import.  
A filename can be supplied, but there is a default ('./mlpconfig').

In [325]:
mlp_net.export_config()

The following code simply checks that a model with weights loaded gives the same result as one with training.

In [326]:
mlp_net2 = MLPNetwork(num_hidden_nodes=100,
                     activation_function=tanh,
                     loss_function='kldiv',
                     verbose=True,
                     learning_rate=0.1)
mlp_net2.import_config()
y_predictions2 = mlp_net.predict(X_test)
print(y_predictions.shape==y_predictions2.shape)